In [4]:
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [6]:
df = sns.load_dataset('penguins')

df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [9]:
df.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,342.000000,342.000000,342.000000,342.000000
mean,43.921930,17.151170,200.915205,4201.754386
std,5.459584,1.974793,14.061714,801.954536
min,32.100000,13.100000,172.000000,2700.000000
25%,39.225000,15.600000,190.000000,3550.000000
50%,44.450000,17.300000,197.000000,4050.000000
75%,48.500000,18.700000,213.000000,4750.000000
max,59.600000,21.500000,231.000000,6300.000000


In [10]:
# Criação de um dicionário de dados

df_dict = pd.DataFrame([
    {
        "variavel": "species",
        "descricao": "Espécie do pinguim ('Adelie', 'Chinstrap', 'Gentoo')",
        "tipo": "qualitativa",
        "subtipo": "nominal",
        "resposta": True
    },{
        "variavel": "island",
        "descricao": "Ilha em que o pinguim foi encontrado ('Torgersen', 'Biscoe', 'Dream')",
        "tipo": "qualitativa",
        "subtipo": "nominal",
        "resposta": False
    },{
        "variavel": "bill_length_mm",
        "descricao": "Comprimento do bico do pinguim em milímetros",
        "tipo": "quantitativa",
        "subtipo": "continua",
        "resposta": False
    },{
        "variavel": "bill_depth_mm",
        "descricao": "Profundidade do bico do pinguim em milímetros",
        "tipo": "quantitativa",
        "subtipo": "continua",
        "resposta": False
    },{
        "variavel": "flipper_length_mm",
        "descricao": "Comprimento da nadadeira do pinguim em milímetros",
        "tipo": "quantitativa",
        "subtipo": "continua",
        "resposta": False
    },{
        "variavel": "body_mass_g",
        "descricao": "Massa corporal do pinguim em gramas",
        "tipo": "quantitativa",
        "subtipo": "continua",
        "resposta": False
    },{
        "variavel": "sex",
        "descricao": "Sexo do pinguim ('Male', 'Female')",
        "tipo": "qualitativa",
        "subtipo": "nominal",
        "resposta": False
    }
])

df_dict

,variavel,descricao,tipo,subtipo,resposta
0,species,"Espécie do pinguim ('Adelie', 'Chinstrap', 'Ge...",qualitativa,nominal,True
1,island,Ilha em que o pinguim foi encontrado ('Torgers...,qualitativa,nominal,False
2,bill_length_mm,Comprimento do bico do pinguim em milímetros,quantitativa,continua,False
3,bill_depth_mm,Profundidade do bico do pinguim em milímetros,quantitativa,continua,False
4,flipper_length_mm,Comprimento da nadadeira do pinguim em milímetros,quantitativa,continua,False
5,body_mass_g,Massa corporal do pinguim em gramas,quantitativa,continua,False
6,sex,"Sexo do pinguim ('Male', 'Female')",qualitativa,nominal,False


In [71]:
# Separar variáveis

variavel_resposta = df_dict.query('resposta == True').variavel.to_list()
variaveis_interesse = df_dict.query('resposta == False').variavel.to_list()

df_X = df[variaveis_interesse]
df_y = df[variavel_resposta]

nominal_features = df_dict.query('subtipo == "nominal" and resposta == False').variavel.to_list()
continuos_features = df_dict.query('subtipo == "continua" and resposta == False').variavel.to_list()

In [72]:
# Tratamento de dados discrepantes

In [77]:
nominal_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False)), # codificação de variáveis
    ('normalization', StandardScaler()), # normalização de variáveis
])

continuos_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # tratamento de dados faltantes
    ('normalization', StandardScaler()), # normalização de variáveis
])

transformer = ColumnTransformer(transformers=[
    ('nominal', nominal_preprocessor, nominal_features),
    ('continuos', continuos_preprocessor, continuos_features),
])

In [76]:
transformer.fit_transform(df_X)

array([[-0.97700842, -0.75075719,  2.36968028, ...,  0.78728939,
        -1.4205409 , -0.56462526],
       [-0.97700842, -0.75075719,  2.36968028, ...,  0.12611415,
        -1.06348546, -0.50201047],
       [-0.97700842, -0.75075719,  2.36968028, ...,  0.43127195,
        -0.42078568, -1.1907732 ],
       ...,
       [ 1.02353263, -0.75075719, -0.42199786, ..., -0.73849962,
         1.50731367,  1.93996648],
       [ 1.02353263, -0.75075719, -0.42199786, ..., -1.19623632,
         0.7932028 ,  1.25120375],
       [ 1.02353263, -0.75075719, -0.42199786, ..., -0.53506109,
         0.86461389,  1.50166293]])

In [85]:
transformer.

AttributeError: 'ColumnTransformer' object has no attribute 'transformers_'